In [2]:
# specify .env path, containing LANGCHAIN_PROJECT, LANGCHAIN_TRACING_V2 ("true"), LANGSMITH_API_KEY, OPENAI_API_KEY
%load_ext dotenv
%dotenv .env 

from langchain_core.messages import HumanMessage
from langgraph_sdk import get_sync_client
from langgraph.pregel.remote import RemoteGraph
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import Command
from llm_plan.workflow import State, build_graph
from urllib.parse import quote
import webbrowser

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [ ]:
# initial state
initial_description = (
    "Two agents are tasked with manipulating blocks arranged in stacks on a table, from one configuration to another. "
    "Each block is uniquely labelled by a letter. Both agents can only interact with blocks at the top of each stack, "
    "and only interact with one block at a time. Additionally, one agent can only interact with vowel blocks, "
    "and the other can only interact with consonant blocks. Initially there are blocks A, B, C, O. A is on the table. "
    "B is on top of A, and C and O are on the table. "
    "The goal is to have A on the table, B on the table, C on top of O, and O on top of B."
    ) # user NL description of planning task
# choose direct or pddl mode. If single agent, must use direct.
initial_state: State = {
    "messages": [HumanMessage(content=initial_description)],
    "multi_agent": True,
    "mode": "direct",
    "refinement_iters": 2, 
    "WSL": True
}

config = {
    "configurable": {"thread_id": "nbk-run-002"},
    "tags": ["notebook_manual"],
    "run_name": "LangGraph-notebook-manual_run",
    "metadata": {"dataset": "dev", "purpose": "test"},
}
# whether to use studio
REMOTE = True

In [16]:
# to view in studio, run langgraph dev then this cell
SERVER = "http://127.0.0.1:2024"
client = get_sync_client(url=SERVER)

# see langgraph.json for available graphs
remote = RemoteGraph("workflow", url=SERVER)

thread = client.threads.create()
config["configurable"]["thread_id"] = thread["thread_id"]

In [17]:
if REMOTE:
    g = remote
else:
    cp = MemorySaver()
    g = build_graph().compile(checkpointer=cp)

try:
    for event in g.stream(initial_state, config=config):
        event_name = next(iter(event))
        print(f"Executing event: {event_name}")
        if event_name == "__interrupt__":
            if REMOTE:
                questions = event["__interrupt__"][0]["value"].get("questions", [])
            else:
                questions = event["__interrupt__"][0].value.get("questions", [])
            if questions:
                print("Clarification questions:")
                for q in questions:
                    print(f"- {q}")
except KeyboardInterrupt:
    print("Streaming interrupted by user.")

Executing event: __interrupt__
Clarification questions:
- Can the agents move blocks directly from one stack to another, or do they need to place them on the table first?
- Are there any restrictions on the number of moves each agent can make?
- Can the agents work simultaneously, or do they need to take turns?
- Are there any specific rules about how the agents should coordinate their actions?


In [18]:
answers = [
    "Don't need to place on table first. But should pick up and place as two separate actions.",
    "No.",
    "They cannot move simultaneously.",
    "No additional rules."
]

In [19]:
# Resume streaming
try:
    for event in g.stream(Command(resume=answers), config=config):
        event_name = next(iter(event))
        print(f"Executing event: {event_name}")
except KeyboardInterrupt:
    print("Streaming interrupted by user.")

Executing event: Oracle
Executing event: Reworder
Executing event: JSON coder
Executing event: Task Environment Constructor
Executing event: Workflow splitter
Executing event: Actor node
Executing event: External solver


In [ ]:
# Resume without streaming
g.invoke(Command(resume=answers), config=config)

In [19]:
def open_studio_for_thread(config: dict, base_url: str="http://127.0.0.1:2024", open_browser: bool=True) -> str:
    """
    Get a LangGraph Studio URL for your local server and current thread, optionally opens in browser
    Args:
        config: the config used to run, must include config['configurable']['thread_id']
        base_url: where the local graph server is (default: http://127.0.0.1:2024)
        open_browser: if True, attempt to open the URL in default browser
    """
    thread_id = (config or {}).get("configurable", {}).get("thread_id")
    if not thread_id:
        raise ValueError("No thread_id found in config['configurable']['thread_id'].")


    studio_base = "https://smith.langchain.com/studio/"
    qs = f"?baseUrl={quote(base_url)}"
    if thread_id:
        qs += f"&threadId={quote(str(thread_id))}"

    url = studio_base + qs
    print("LangGraph Studio link:")
    print(url)
    print("\nThread ID:", thread_id)

    if open_browser:
        try:
            webbrowser.open(url)
        except Exception as e:
            print("Could not auto-open browser:", e)

    return url

In [11]:
open_studio_for_thread(config)

LangGraph Studio link:
https://smith.langchain.com/studio/?baseUrl=http%3A//127.0.0.1%3A2024&threadId=nbk-run-002

Thread ID: nbk-run-002


'https://smith.langchain.com/studio/?baseUrl=http%3A//127.0.0.1%3A2024&threadId=nbk-run-002'

In [21]:
# Inspect any field of the final graph state
field = "messages"
final_state = remote.get_state({"configurable": {"thread_id": config["configurable"]["thread_id"]}}).values[field]
final_state

[{'content': 'Two agents are tasked with manipulating blocks arranged in stacks on a table, from one configuration to another. Each block is uniquely labelled by a letter. Both agents can only interact with blocks at the top of each stack, and only interact with one block at a time. Additionally, one agent can only interact with vowel blocks, and the other can only interact with consonant blocks. Initially there are blocks A, B, C, O. A is on the table. B is on top of A, and C and O are on the table. The goal is to have A on the table, B on the table, C on top of O, and O on top of B.',
  'additional_kwargs': {},
  'response_metadata': {},
  'type': 'human',
  'name': None,
  'id': '11383e91-bcea-412c-96b2-3adc0a59c360',
  'example': False},
 {'content': 'I need to clarify a few points to ensure there is enough information to make a plan:\n\n1. Can the agents move blocks directly from one stack to another, or do they need to place them on the table first?\n2. Are there any restrictions